In [ ]:
# Run this block before anything else

%env PYTHONPATH=
!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!rm ./Miniconda3-4.5.4-Linux-x86_64.sh
!conda install -q -y --prefix /usr/local python=3.7 cartopy
import sys
sys.path.append('/usr/local/lib/python3.7/site-packages')
%matplotlib inline

import scipy.io.netcdf as sp
import matplotlib.pyplot as plt
import numpy as np

!pip install cartopy
import cartopy.crs as ccrs
import cartopy.feature as cf
from cartopy.util import add_cyclic_point

from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# This script creates a global contour map of the mean RSR values at each location

print('here 1')

# Open the dataset file
fileObj = sp.NetCDFFile('/content/drive/Shared drives/Research Project Data/CERES_EBAF-TOA_Ed4.1_Subset_200003-202111.nc', 'r')
print('here 2')

# Make variables to store all the RSR, latitude, and longitude values
RSR = fileObj.variables['toa_sw_all_mon'][:]
lat = fileObj.variables['lat'][:]
lon = fileObj.variables['lon'][:]
print('here 3')

# Calculate how many full years of data is in the dataset
numYears = RSR.shape[0] // 12
print('here 4')

# Take a subset of the data in which the extra months at the end that are not part of a full year are excluded
fullYearRSR = RSR[0:numYears * 12, :, :]
print('here 5')

# Average the data at each location across all months
means = fullYearRSR.mean(axis=0)
print('here 6')

# Fix problem with 0 longitude missing data
means, lon = add_cyclic_point(means, coord=lon)
print('here 7')

# Set up a global map using the Plate Carree projection and add coastlines
axes = plt.axes(projection=ccrs.PlateCarree())
axes.add_feature(cf.COASTLINE, linewidth=0.5)
print('here 8')

# Overlay the contours onto the map based on the mean RSR values at each location. Include a color bar and add a title
plt.contourf(lon[:], lat[:], means[:,:], levels=12, cmap='bone')
plt.colorbar(orientation='horizontal')
plt.title('Mean RSR (W/m^2), Mar 2000 - Feb 2020')
print('here 9')

# Save a PNG image of the map
plt.savefig('/content/drive/My Drive/Research Project/RSR_Map.png', bbox_inches='tight', dpi=200)
print('here 10')

# projection options: https://scitools.org.uk/cartopy/docs/latest/crs/projections.html
# cmap options: https://matplotlib.org/stable/tutorials/colors/colormaps.html